In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import timeit
#import shap
from sklearn.metrics import r2_score, mean_absolute_percentage_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.model_selection import train_test_split

___

# Read-in and One-Hot Encoding

In [2]:
df1 = pd.read_csv("data/arrival_time_COMPLETE.csv")
df2 = pd.read_csv("data/negative_duration_COMPLETE.csv")
df3 = pd.read_csv("data/negative_peak_time_COMPLETE.csv")
df4 = pd.read_csv("data/negative_pressure_COMPLETE.csv")
df5 = pd.read_csv("data/positive_duration_COMPLETE.csv")
df6 = pd.read_csv("data/positive_impulse_COMPLETE.csv")
df7 = pd.read_csv("data/positive_peak_time_COMPLETE.csv")
df8 = pd.read_csv("data/positive_pressure_COMPLETE.csv")

df1.head(5)

,ID,Tank Failure Pressure (bar),Liquid Ratio (%),Tank Width (m),Tank Length (m),Tank Height (m),Height of BLEVE (m),Tank Height with Gas (m),Vapour Temerature (K),Liquid Temerature (K),Status,Stand-off Distance,Target
0,B1,24.5,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,Subcooled,5.0,0.010208
1,B1,24.5,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,Subcooled,6.0,0.012350
2,B1,24.5,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,Subcooled,7.0,0.014577
3,B1,24.5,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,Subcooled,8.0,0.016878
4,B1,24.5,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,Subcooled,9.0,0.019250


In [24]:
# Label encoding 'Status' feature into 0 and 1 
# 0 for Subcooled and 1 for Superheated
# Doing Similarly for ID (Do we need dummy encoding ??)

from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()

df5['Status'] = LE.fit_transform(df5['Status'])
df5['ID'] = LE.fit_transform(df5['ID'])
df5

,ID,Tank Failure Pressure (bar),Liquid Ratio (%),Tank Width (m),Tank Length (m),Tank Height (m),Height of BLEVE (m),Tank Height with Gas (m),Vapour Temerature (K),Liquid Temerature (K),Status,Stand-off Distance,Target
0,0,24.50000,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,0,5.0,0.006817
1,0,24.50000,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,0,6.0,0.007302
2,0,24.50000,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,0,7.0,0.007816
3,0,24.50000,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,0,8.0,0.008326
4,0,24.50000,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,0,9.0,0.008817
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35995,946,11.40239,0.442321,1.4,2.8,1.2,1.4,0.8,388.2,366.7,1,36.0,NaN
35996,946,11.40239,0.442321,1.4,2.8,1.2,1.4,0.8,388.2,366.7,1,37.0,NaN
35997,946,11.40239,0.442321,1.4,2.8,1.2,1.4,0.8,388.2,366.7,1,38.0,NaN
35998,946,11.40239,0.442321,1.4,2.8,1.2,1.4,0.8,388.2,366.7,1,39.0,NaN


In [25]:
X = df5.drop('Target', axis=1)[:28800]
X

,ID,Tank Failure Pressure (bar),Liquid Ratio (%),Tank Width (m),Tank Length (m),Tank Height (m),Height of BLEVE (m),Tank Height with Gas (m),Vapour Temerature (K),Liquid Temerature (K),Status,Stand-off Distance
0,0,24.50000,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,0,5.0
1,0,24.50000,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,0,6.0
2,0,24.50000,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,0,7.0
3,0,24.50000,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,0,8.0
4,0,24.50000,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...
28795,724,33.17377,0.372041,1.0,2.2,0.6,0.2,0.4,312.7,318.2,0,36.0
28796,724,33.17377,0.372041,1.0,2.2,0.6,0.2,0.4,312.7,318.2,0,37.0
28797,724,33.17377,0.372041,1.0,2.2,0.6,0.2,0.4,312.7,318.2,0,38.0
28798,724,33.17377,0.372041,1.0,2.2,0.6,0.2,0.4,312.7,318.2,0,39.0


In [26]:
y5 = df5['Target'][:28800]
y5

0        0.006817
1        0.007302
2        0.007816
3        0.008326
4        0.008817
           ...   
28795    0.012178
28796    0.012275
28797    0.012374
28798    0.012477
28799    0.012573
Name: Target, Length: 28800, dtype: float64

In [27]:
y1 = df1['Target'][:28800]
y2 = df2['Target'][:28800]
y3 = df3['Target'][:28800]
y4 = df4['Target'][:28800]
y6 = df6['Target'][:28800]
y7 = df7['Target'][:28800]
y8 = df8['Target'][:28800]

___

# Train-Test Set Split

In [28]:
X_traindf, X_testdf, y1_train, y1_test = train_test_split(X, y1, test_size=0.25, random_state=42)
print(X_traindf.shape)
print(X_testdf.shape)

(21600, 12)
(7200, 12)


In [29]:
y2_train, y2_test = train_test_split(y2, test_size=0.25, random_state=42)
y3_train, y3_test = train_test_split(y3, test_size=0.25, random_state=42)
y4_train, y4_test = train_test_split(y4, test_size=0.25, random_state=42)
y5_train, y5_test = train_test_split(y5, test_size=0.25, random_state=42)
y6_train, y6_test = train_test_split(y6, test_size=0.25, random_state=42)
y7_train, y7_test = train_test_split(y7, test_size=0.25, random_state=42)
y8_train, y8_test = train_test_split(y8, test_size=0.25, random_state=42)

In [30]:
y8_test

18932    0.080268
21280    0.399188
27880    0.184898
15692    0.070768
25416    1.603528
           ...   
18078    0.233269
3566     0.238918
4129     0.092671
9939     1.170312
11028    0.047712
Name: Target, Length: 7200, dtype: float64

___

# Standardization and Power Transform

In [31]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer

In [32]:
# Processing both X_train and X_test using standardization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_traindf)
X_test = scaler.transform(X_testdf)

In [34]:
# Double check if it succeeded
# df_stdscal = pd.DataFrame(X_train)
# df_stdscal.hist(figsize = (20,20), bins=10)

___

# ANN

In [35]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

torch.backends.cudnn.version() , torch.backends.cudnn.enabled

(8500, True)

In [96]:
# convert data to torch.FloatTensor
X_train_torch = torch.from_numpy(X_train.astype(np.float32))
X_test_torch = torch.from_numpy(X_test.astype(np.float32))



y1_train_torch = torch.from_numpy(y1_train.values.astype(np.float32).reshape(-1, 1))
y1_test_torch = torch.from_numpy(y1_test.values.astype(np.float32).reshape(-1, 1))

y2_train_torch = torch.from_numpy(y2_train.values.astype(np.float32).reshape(-1, 1))
y2_test_torch = torch.from_numpy(y2_test.values.astype(np.float32).reshape(-1, 1))

y3_train_torch = torch.from_numpy(y3_train.values.astype(np.float32).reshape(-1, 1))
y3_test_torch = torch.from_numpy(y3_test.values.astype(np.float32).reshape(-1, 1))

y4_train_torch = torch.from_numpy(y4_train.values.astype(np.float32).reshape(-1, 1))
y4_test_torch = torch.from_numpy(y4_test.values.astype(np.float32).reshape(-1, 1))

y5_train_torch = torch.from_numpy(y5_train.values.astype(np.float32).reshape(-1, 1))
y5_test_torch = torch.from_numpy(y5_test.values.astype(np.float32).reshape(-1, 1))

y6_train_torch = torch.from_numpy(y6_train.values.astype(np.float32).reshape(-1, 1))
y6_test_torch = torch.from_numpy(y6_test.values.astype(np.float32).reshape(-1, 1))

y7_train_torch = torch.from_numpy(y7_train.values.astype(np.float32).reshape(-1, 1))
y7_test_torch = torch.from_numpy(y7_test.values.astype(np.float32).reshape(-1, 1))

y8_train_torch = torch.from_numpy(y8_train.values.astype(np.float32).reshape(-1, 1))
y8_test_torch = torch.from_numpy(y8_test.values.astype(np.float32).reshape(-1, 1))

print(X_train_torch)

tensor([[-0.0924,  0.1781, -0.9747,  ..., -0.7230, -0.7424, -1.4851],
        [ 0.9252, -0.5207,  0.5219,  ..., -0.3853,  1.3470, -1.4851],
        [-1.3624,  1.3911,  1.5275,  ...,  1.1182, -0.7424,  1.3998],
        ...,
        [-0.9952, -1.6514,  0.5764,  ...,  0.7199,  1.3470,  1.3998],
        [-0.1498,  0.9835,  0.0136,  ..., -0.4908, -0.7424,  0.9190],
        [ 0.5694,  1.3715,  0.0745,  ...,  0.5537,  1.3470, -1.4851]])


In [99]:
y8_test_torch

tensor([[0.0803],
        [0.3992],
        [0.1849],
        ...,
        [0.0927],
        [1.1703],
        [0.0477]])

In [100]:
loader = [[1,2,3], [4,5,6]]

In [101]:
train_data = []
for i in range(len(X_train_torch)):
   train_data.append([X_train_torch[i],
                      y1_train_torch[i], 
                      y2_train_torch[i],
                      y3_train_torch[i],
                      y4_train_torch[i],
                      y5_train_torch[i],
                      y6_train_torch[i],
                      y7_train_torch[i],
                      y8_train_torch[i]])

test_data = []
for i in range(len(X_test_torch)):
   test_data.append([X_test_torch[i], 
                     y1_test_torch[i],
                     y2_test_torch[i],
                     y3_test_torch[i],
                     y4_test_torch[i],
                     y5_test_torch[i],
                     y6_test_torch[i],
                     y7_test_torch[i],
                     y8_test_torch[i]])


# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=512,
    num_workers=6)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=512,
    num_workers=6)

In [102]:
## Define the NN architecture
## NN with 3 hidden layer, s=[26, 256, 256, 256, 1]

class BleveNet(nn.Module):
    def __init__(self):
        super(BleveNet, self).__init__()
        # The first hidden layer has 256 neurons
        self.fc1 = nn.Linear(X_train_torch.shape[1], 256)
        # The second hidden layer has 256 neurons
        self.fc2 = nn.Linear(256, 256)
        # The third hidden layer has 256 neurons
        self.fc3 = nn.Linear(256, 256)
        # The final layer has 1 output neuron
        self.fc4 = nn.Linear(256, 1)



        # Define proportion or neurons to dropout
        self.dropout = nn.Dropout(0.1)

        # Mish activation
        self.mish = nn.Mish()


    def forward(self, x):
        # add first fully connected layer
        x = self.fc1(x)
        x = self.mish(x)
        x = self.dropout(x)
        # add second fully connected layer
        x = self.fc2(x)
        x = self.mish(x)
        x = self.dropout(x)
        # add third fully connected layer
        x = self.fc3(x)
        x = self.mish(x)
        x = self.dropout(x)
        # add final fully connected layers, separated in 8 different outputs
        output1 = self.fc4(x)
        output2 = self.fc5(x)
        output3 = self.fc4(x)
        output4 = self.fc4(x)
        output5 = self.fc4(x)
        output6 = self.fc4(x)
        output7 = self.fc4(x)
        output8 = self.fc4(x)
        return output1, output2, output3, output4, output5, output6, output7, output8

# initialize the NN
model = BleveNet()
print(model)

BleveNet(
  (fc1): Linear(in_features=12, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (mish): Mish()
)


In [103]:
criterion = nn.HuberLoss()        # This is the best loss function for my model
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=1e-5)     # This is the best optimizer for my model 

# Check for GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [104]:
n_epochs = 200 

model.to(device)    # bring the model to gpu
model.train()       # prep model for training

train_losses = []
for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################
    for data, target1, target2, target3, target4, target5, target6, target7, target8 in train_loader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        #bring data and target to gpu
        data, target1 = data.to(device), target1.to(device)
        target2, target3 = target2.to(device), target3.to(device)
        target4, target5 = target4.to(device), target5.to(device)
        target6, target7 = target6.to(device), target7.to(device)
        target8 = target8.to(device)
        # forward pass: compute predicted outputs by passing inputs to the model
        output1, output2, output3, output4, output5, output6, output7, output8 = model(data)
        # calculate the loss
        loss1 = criterion(output1, target1)
        loss2 = criterion(output2, target2)
        loss3 = criterion(output3, target3)
        loss4 = criterion(output4, target4)
        loss5 = criterion(output5, target5)
        loss6 = criterion(output6, target6)
        loss7 = criterion(output7, target7)
        loss8 = criterion(output8, target8)
        loss = loss1 + loss2 + loss3 + loss4 + loss5 + loss6 + loss7 + loss8
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        
    # print training statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    train_losses.append(train_loss)

    print('Epoch: {}/{} \tTraining Loss: {:.6f}'.format(
        epoch+1,
        n_epochs, 
        train_loss
        ))

Epoch: 1/200 	Training Loss: 84.381816
Epoch: 2/200 	Training Loss: 84.328350
Epoch: 3/200 	Training Loss: 84.327960
Epoch: 4/200 	Training Loss: 84.327797
Epoch: 5/200 	Training Loss: 84.327705
Epoch: 6/200 	Training Loss: 84.327631
Epoch: 7/200 	Training Loss: 84.327581
Epoch: 8/200 	Training Loss: 84.327550
Epoch: 9/200 	Training Loss: 84.327513
Epoch: 10/200 	Training Loss: 84.327472
Epoch: 11/200 	Training Loss: 84.327449
Epoch: 12/200 	Training Loss: 84.327436
Epoch: 13/200 	Training Loss: 84.327410
Epoch: 14/200 	Training Loss: 84.327406
Epoch: 15/200 	Training Loss: 84.327397
Epoch: 16/200 	Training Loss: 84.327387
Epoch: 17/200 	Training Loss: 84.327368
Epoch: 18/200 	Training Loss: 84.327361
Epoch: 19/200 	Training Loss: 84.327352
Epoch: 20/200 	Training Loss: 84.327360
Epoch: 21/200 	Training Loss: 84.327338
Epoch: 22/200 	Training Loss: 84.327330
Epoch: 23/200 	Training Loss: 84.327332
Epoch: 24/200 	Training Loss: 84.327322
Epoch: 25/200 	Training Loss: 84.327319
Epoch: 26

In [ ]:
with torch.no_grad():
    output1, output2, output3, output4, output5, output6, output7, output8 = model(X_test_torch)
    y1_pred, y2_pred = output1.numpy(), output2.numpy()

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
torch.manual_seed(42)

# Creating the dataset class
class Data(Dataset):
    # Constructor
    def __init__(self):
        self.x = torch.zeros(40, 2)
        self.x[:, 0] = torch.arange(-2, 2, 0.1)
        self.x[:, 1] = torch.arange(-2, 2, 0.1)
        w = torch.tensor([[1.0, 2.0], [2.0, 4.0]])
        b = 1
        func = torch.mm(self.x, w) + b    
        self.y = func + 0.2 * torch.randn((self.x.shape[0],1))
        self.len = self.x.shape[0]
    # Getter
    def __getitem__(self, idx):          
        return self.x[idx], self.y[idx] 
    # getting data length
    def __len__(self):
        return self.len

In [2]:
# Creating dataset object
data_set = Data()

In [7]:
data_set.__getitem__(39)

(tensor([1.9000, 1.9000]), tensor([ 6.4428, 12.1428]))

In [13]:
# Creating the dataloader
train_loader = DataLoader(dataset=data_set, batch_size=2)
epochs = 2
for epoch in range(epochs):
    for x,y in train_loader:
        print(y)


tensor([[ -4.6146, -10.6146],
        [ -4.4025, -10.1025]])
tensor([[-4.2199, -9.6199],
        [-4.5211, -9.6211]])
tensor([[-3.6643, -8.4643],
        [-3.7469, -8.2469]])
tensor([[-3.2086, -7.4086],
        [-3.2209, -7.1209]])
tensor([[-2.7504, -6.3504],
        [-1.9703, -5.2703]])
tensor([[-2.0785, -5.0785],
        [-1.9807, -4.6807]])
tensor([[-1.5456, -3.9456],
        [-1.2119, -3.3119]])
tensor([[-0.9538, -2.7538],
        [-0.3475, -1.8475]])
tensor([[ 0.1285, -1.0715],
        [ 0.0681, -0.8319]])
tensor([[ 0.3005, -0.2995],
        [ 0.7879,  0.4879]])
tensor([[0.8484, 0.8484],
        [1.5157, 1.8157]])
tensor([[1.7602, 2.3602],
        [2.2361, 3.1361]])
tensor([[2.2070, 3.4070],
        [2.5642, 4.0642]])
tensor([[3.1147, 4.9147],
        [2.9309, 5.0309]])
tensor([[3.6625, 6.0625],
        [3.8374, 6.5374]])
tensor([[3.7822, 6.7822],
        [4.2289, 7.5289]])
tensor([[4.3164, 7.9164],
        [5.0793, 8.9793]])
tensor([[ 5.2100,  9.4100],
        [ 5.9533, 10.4533]]